## Compensation Data Random Exploration

In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn.linear_model as skl_lm
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import Ridge, RidgeCV
from sklearn.cross_decomposition import PLSRegression
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

import statsmodels.api as sm

import seaborn as sns

pd.set_option('display.max_rows', 100)
plt.rcParams["figure.figsize"] = (20,10)

In [7]:
comp = pd.read_csv('data/prepped_comp_data.csv', index_col='entry_id')
# comp = pd.read_csv('data/clean_comp_data.csv', index_col="entry_id")
comp

,log_total_comp,log_salary,log_stock,log_bonus,scaled_years_experience,scaled_years_company,company[AT&T],company[Accenture],company[Adobe],company[Airbnb],...,tag[Full Stack],tag[ML / AI],tag[Mobile (iOS + Android)],tag[Networking],tag[Operating Systems],tag[Security],tag[Site Reliability (SRE)],tag[Testing (SDET)],tag[Web Development (Front-End)],tag[iOS]
entry_id,,,,,,,,,,,,,,,,,,,,,
4,6.359576,5.192962,5.959718,2.197336,0.712125,-0.123292,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,5.153297,4.787500,-6.907755,3.970311,0.896446,-0.467863,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,5.247029,4.700489,4.382039,-6.907755,-0.578121,0.221278,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,5.049862,4.905282,2.079567,2.565026,-0.393800,0.565848,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,5.303310,5.056252,3.258135,3.332240,1.080767,1.254988,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30133,4.744941,4.744941,-6.907755,-6.907755,-0.025158,-0.467863,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30139,5.913506,5.231114,5.010642,3.496538,0.712125,1.254988,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
30140,5.356591,5.176155,-6.907755,3.555377,-0.762442,-0.812433,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
RANDOM_STATE = 721
X_train, X_test, _, _ = train_test_split(comp, comp, test_size=0.1, random_state=RANDOM_STATE)

y_total_comp = np.exp(X_train.log_total_comp)
y_salary = np.exp(X_train.log_salary)
y_stock = np.exp(X_train.log_stock)
y_bonus = np.exp(X_train.log_bonus)
X = X_train.drop(columns=['log_total_comp', 'log_salary', 'log_stock', 'log_bonus'])

y_test_total_comp = np.exp(X_test.log_total_comp)
y_test_salary = np.exp(X_test.log_salary)
y_test_stock = np.exp(X_test.log_stock)
y_test_bonus = np.exp(X_test.log_bonus)
X_test = X_test.drop(columns=['log_total_comp', 'log_salary', 'log_stock', 'log_bonus'])

X
X_test

,scaled_years_experience,scaled_years_company,company[AT&T],company[Accenture],company[Adobe],company[Airbnb],company[Amazon],company[American Express],company[Andela],company[Apple],...,tag[Full Stack],tag[ML / AI],tag[Mobile (iOS + Android)],tag[Networking],tag[Operating Systems],tag[Security],tag[Site Reliability (SRE)],tag[Testing (SDET)],tag[Web Development (Front-End)],tag[iOS]
entry_id,,,,,,,,,,,,,,,,,,,,,
1042,0.343483,0.221278,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6488,-0.393800,0.565848,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21860,-0.762442,-0.123292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7900,-0.209479,0.910418,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4392,-0.393800,0.565848,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28116,-0.762442,-0.123292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13550,-0.025158,-0.467863,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8130,0.527804,-0.123292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,scaled_years_experience,scaled_years_company,company[AT&T],company[Accenture],company[Adobe],company[Airbnb],company[Amazon],company[American Express],company[Andela],company[Apple],...,tag[Full Stack],tag[ML / AI],tag[Mobile (iOS + Android)],tag[Networking],tag[Operating Systems],tag[Security],tag[Site Reliability (SRE)],tag[Testing (SDET)],tag[Web Development (Front-End)],tag[iOS]
entry_id,,,,,,,,,,,,,,,,,,,,,
23789,-0.578121,-0.812433,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11706,0.712125,1.254988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
310,-0.578121,-0.467863,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
815,-1.131083,-0.812433,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5982,0.343483,-0.812433,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25211,-0.578121,-0.812433,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25710,-0.578121,-0.467863,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13958,0.896446,-0.812433,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
mlp = MLPRegressor(hidden_layer_sizes=(1000,), activation='logistic',
                    solver='sgd', tol=1e-4, random_state=721,
                    learning_rate_init=.001, max_iter=2000, verbose=True)
mlp.fit(X, y_salary)

Iteration 1, loss = 1250.08486816
Iteration 2, loss = 808.35344969
Iteration 3, loss = 748.70151739
Iteration 4, loss = 699.81585055
Iteration 5, loss = 651.47939347
Iteration 6, loss = 599.83707474
Iteration 7, loss = 544.58006350
Iteration 8, loss = 489.73669338
Iteration 9, loss = 440.02084968
Iteration 10, loss = 402.21579508
Iteration 11, loss = 375.19617044
Iteration 12, loss = 358.34440871
Iteration 13, loss = 346.92191419
Iteration 14, loss = 340.64862619
Iteration 15, loss = 332.89580036
Iteration 16, loss = 328.77218566
Iteration 17, loss = 324.26292841
Iteration 18, loss = 321.02384763
Iteration 19, loss = 318.17892250
Iteration 20, loss = 315.68408355
Iteration 21, loss = 313.90154953
Iteration 22, loss = 313.02639376
Iteration 23, loss = 310.81566149
Iteration 24, loss = 309.54808634
Iteration 25, loss = 308.44387568
Iteration 26, loss = 306.42885631
Iteration 27, loss = 306.10608928
Iteration 28, loss = 304.52797426
Iteration 29, loss = 303.79179949
Iteration 30, loss = 3

/Users/simonayzman/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPRegressor(activation='logistic', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(1000,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=2000,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=721, shuffle=True, solver='sgd',
             tol=0.0001, validation_fraction=0.1, verbose=True,
             warm_start=False)

In [10]:
mlp2 = MLPRegressor(hidden_layer_sizes=(700,), activation='logistic',
                    solver='sgd', tol=1e-4, random_state=721,
                    learning_rate_init=.005, max_iter=2000, verbose=True)
mlp2.fit(X, y_salary)

Iteration 1, loss = 990.16060146
Iteration 2, loss = 478.11101452
Iteration 3, loss = 399.41030180
Iteration 4, loss = 366.41076284
Iteration 5, loss = 345.63608810
Iteration 6, loss = 333.14138800
Iteration 7, loss = 325.64025704
Iteration 8, loss = 318.41680508
Iteration 9, loss = 312.33435672


/Users/simonayzman/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPRegressor(activation='logistic', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(700,), learning_rate='constant',
             learning_rate_init=0.005, max_fun=15000, max_iter=2000,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=721, shuffle=True, solver='sgd',
             tol=0.0001, validation_fraction=0.1, verbose=True,
             warm_start=False)

In [11]:
y_train_pred = mlp2.predict(X_train).flatten()
y_train_pred
test_error = np.sqrt(mean_squared_error(y_salary, y_train_pred))
test_error

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 191 is different from 195)

In [ ]:
mlpr = MLPRegressor(max_iter=7000, tol=1e-4, verbose=True)

param_list = {
    "hidden_layer_sizes": [(50,), (200,), (500,)],
    #"activation": ["identity", "logistic", "tanh", "relu"],
    #"solver": ["lbfgs", "sgd", "adam"],
    "alpha": [0.00005, 0.0005],
}

gridCV = GridSearchCV(estimator=mlpr, param_grid=param_list, n_jobs=-1, verbose=10)

gridCV.fit(X, y_salary)